- Update: 2023.02.25

# 0. 配置参数

- `dict_menu`：目录文件的路径
- `out`：保存结果的路径

In [1]:
dict_menu = {'file': '../../file/cardone-throttle_body-menu.xlsx',
             'head': 1}

out = '../../file/cardone-throttle_body-content.xlsx'

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel(dict_menu['file'], header=dict_menu['head']-1)

df_menu

,标题,品牌号,车型,链接
0,Fuel Injection Throttle Body - 6E-4204,6E-4204,"Mazda 3 2005-04, 5 2007-06, 6 2005-03",https://www.cardone.com/product/fuel-injection...
1,Fuel Injection Throttle Body - 6E-6011,6E-6011,Ford F-150 2008-05,https://www.cardone.com/product/fuel-injection...
2,Fuel Injection Throttle Body - 6E-3019,6E-3019,"Buick Allure 2010, LaCrosse 2010\nCadillac CTS...",https://www.cardone.com/product/fuel-injection...
3,Fuel Injection Throttle Body - 6E-3015,6E-3015,"Cadillac Catera 2001-99, CTS 2004-03",https://www.cardone.com/product/fuel-injection...
4,Fuel Injection Throttle Body - 6E-8014,6E-8014,"Scion iM 2016\nToyota Corolla 2019-11, Corolla...",https://www.cardone.com/product/fuel-injection...
...,...,...,...,...
416,Fuel Injection Throttle Body - 67-6011,67-6011,Ford F-150 2008-05,https://www.cardone.com/product/fuel-injection...
417,Fuel Injection Throttle Body - 67-7017,67-7017,Fiat 500 2017-12,https://www.cardone.com/product/fuel-injection...
418,Fuel Injection Throttle Body - 67-6014,67-6014,"Ford Focus 2011-08, Transit Connect 2013-10",https://www.cardone.com/product/fuel-injection...
419,Fuel Injection Throttle Body - 67-3015,67-3015,"Cadillac Catera 2001-99, CTS 2004-03",https://www.cardone.com/product/fuel-injection...


In [4]:
from selenium.webdriver import Chrome
import datetime

In [5]:
browser = Chrome()
browser.set_page_load_timeout(60)

print(f'开始爬取时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

开始爬取时间：2023:02:25 09:16:53


In [6]:
from selenium.webdriver.common.by import By
import time

In [7]:
col_href = []
col_oe = []
for i in range(len(df_menu)):
    print(i+1)
    browser.get(df_menu.loc[i, '链接'] + '#tab-specs')
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)
    browser.execute_script('window.scrollTo(0, -document.body.scrollHeight)')
    time.sleep(0.5)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    
    href = browser.find_element(by=By.CLASS_NAME, value='productView-img-container').find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    col_href.append(href)
    # print(href)
    list_oe = []
    list_div = browser.find_element(by=By.CLASS_NAME, value='oe-nums').find_elements(by=By.CLASS_NAME, value='productView-table-data')
    for div in list_div:
        list_str = div.text.split(',')
        for string in list_str:
            list_oe.append(string.strip())
    list_oe = set(list_oe)
    oe = ';'.join(list_oe)
    col_oe.append(oe)
    # print(oe)
    # print()

171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420


In [13]:
df_menu['图片链接'] = col_href
df_menu['OE号'] = col_oe

df_content = df_menu[['标题', '品牌号', 'OE号', '车型', '链接', '图片链接']]

C:\Users\YangTeng\AppData\Local\Temp\ipykernel_9652\2238865004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_menu['图片链接'] = col_href
C:\Users\YangTeng\AppData\Local\Temp\ipykernel_9652\2238865004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_menu['OE号'] = col_oe


In [14]:
browser.quit()
print(f'爬取结束时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

爬取结束时间：2023:02:25 09:58:50


In [15]:
df_content.to_excel(out, index=False)